## Criação de um modelo utilizando o Naive Bayes

Posteriormente sera implementado as funcionalidades do MLFlow

Criar experimentos para avaliação

In [6]:
!pip install mlflow
!pip install scikit-learn
!pip install yellowbrick

import pandas as pd

from sklearn.model_selection import train_test_split 
# Biblioteca que será usada para train test
from sklearn.naive_bayes import GaussianNB
# Algoritmo Naive Bayes para dados de distribuição Gaussiana
from sklearn.metrics import * 
# Performance

import matplotlib.pyplot as plt 
# Gráficos

import mlflow
import mlflow.sklearn 
# Integração MLFlow


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\Rafael\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\Rafael\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\Rafael\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
# Utilizando o Pandas, criamos o df "credit" e obtemos o número de colunas e linhas
credit = pd.read_csv('E:/WorkSpace/MachineLearningModel/Files/Credit.csv')
credit.shape

(1000, 21)

In [8]:
# Visualização do df
credit.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


In [9]:
# Utilizando o Pandas, transformamos os dados categoricos para numeros, 
# Permitindo que o sklearn possa criar nosso modelo

for col in credit.columns:
    if credit[col].dtype == 'object': # Verifica o tipo da coluna
        credit[col] = credit[col].astype('category').cat.codes # Conversão para tipo "category"
        
credit.head(10)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,2,6,1,7,1169,0,3,4,3,2,...,2,67,1,1,2,3,1,1,1,1
1,1,48,3,7,5951,3,0,2,0,2,...,2,22,1,1,1,3,1,0,1,0
2,0,12,1,4,2096,3,1,2,3,2,...,2,49,1,1,1,2,2,0,1,1
3,2,42,3,5,7882,3,1,2,3,1,...,0,45,1,0,1,3,2,0,1,1
4,2,24,2,1,4870,3,0,3,3,2,...,1,53,1,0,2,3,2,0,1,0
5,0,36,3,4,9055,0,0,2,3,2,...,1,35,1,0,1,2,2,1,1,1
6,0,24,3,5,2835,2,3,3,3,2,...,0,53,1,1,1,3,1,0,1,1
7,1,36,3,2,6948,3,0,2,3,2,...,3,35,1,2,1,0,1,1,1,1
8,0,12,3,7,3059,4,1,2,1,2,...,2,61,1,1,1,2,1,0,1,1
9,1,30,1,1,5234,3,4,4,2,2,...,3,28,1,1,2,0,1,0,1,0


In [10]:
# Seleção das linhas e as colunas de 0 a 19 do df credit 
forecasters = credit.iloc[:,0:20].values
# Conversão de um numpy array com os valores
numpy = credit.iloc[:,20].values


forecasters

array([[ 2,  6,  1, ...,  1,  1,  1],
       [ 1, 48,  3, ...,  1,  0,  1],
       [ 0, 12,  1, ...,  2,  0,  1],
       ...,
       [ 0, 12,  3, ...,  1,  0,  1],
       [ 2, 45,  3, ...,  1,  1,  1],
       [ 1, 45,  1, ...,  1,  0,  1]], dtype=int64)

## Treino e Teste

Sendo 70% Treino e 30% Teste

In [11]:
# forecasters 
# Variaveis independentes para treinamento
# Variaveis independentes para testes


# numpy
# Variavel dependentes para treinamento
# Variavel dependentes para testes
trainingX, TestResultX, trainingY, TestResultY = train_test_split(forecasters, numpy, test_size = 0.3, random_state = 123)

## Criação do Modelo

In [12]:
# Cria o objeto
naive_bayes_object = GaussianNB()
# Criando o modelo passando as variáveis independentes e minha variável dependente
naive_bayes_object.fit(trainingX, trainingY)

# Criando previsões
forecasters = naive_bayes_object.predict(TestResultX)

# Criando métricas

# Calcula a acurácia do modelo
accuracy_model = accuracy_score(TestResultY, forecasters)
print(f"Acurácia do Modelo: {accuracy_model}")

# Calcula o relatório de classificação
print("\nRelatório de Classificação:\n")
print(classification_report(TestResultY, forecasters))

# Matriz de confusão
print("Matriz de Confusão:")
print(confusion_matrix(TestResultY, forecasters))


Acurácia do Modelo: 0.6966666666666667

Relatório de Classificação:

              precision    recall  f1-score   support

           0       0.54      0.55      0.55       100
           1       0.77      0.77      0.77       200

    accuracy                           0.70       300
   macro avg       0.66      0.66      0.66       300
weighted avg       0.70      0.70      0.70       300

Matriz de Confusão:
[[ 55  45]
 [ 46 154]]
